In [50]:
N, K = map(int, input().split())
P = list(map(int, input().split()))

 15 3
 3 14 15 9 2 6 5 13 1 7 10 11 8 12 4


In [51]:
import math
from bisect import bisect_left, bisect_right
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedSet(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedSet from iterable. / O(N) if sorted and unique / O(N log N)"
        a = list(a)
        if not all(a[i] < a[i + 1] for i in range(len(a) - 1)):
            a = sorted(set(a))
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedSet" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def add(self, x: T) -> bool:
        "Add an element and return True if added. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return True
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i != len(a) and a[i] == x: return False
        a.insert(i, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()
        return True

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True
    
    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError
    
    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return 

In [69]:
# AC(辞書を使う)

from collections import deque

d = {}
ans = [-1]*N

s = SortedSet([])
for turn, p in enumerate(P):
    tmp = s.gt(p)
    if tmp == None:
        s.add(p)
        d[p] = deque([p])
        if K == 1:
            ans[p-1] = turn+1
            s.discard(p)
    else:
        s.discard(tmp)
        s.add(p)
        d[tmp].append(p)
        d[p] = d[tmp]
        del d[tmp]
        if len(d[p]) == K:
            while (d[p]):
                t = d[p].pop()
                ans[t-1] = turn+1
            s.discard(p)
for a in ans:
    print (a)

9
9
9
15
15
6
-1
-1
6
-1
-1
-1
-1
6
15


In [54]:
# AC(pre配列を使う)

cards = [0]*N
pre = [0]*N
ans = [-1]*N

s = SortedSet([])
for turn, p in enumerate(P):
    tmp = s.gt(p)
    if tmp == None:
        s.add(p)
        cards[p-1] = 1
        if K == 1:
            ans[p-1] = turn+1
            s.discard(p)
    else:
        s.discard(tmp)
        s.add(p)
        cards[tmp-1] += 1
        cards[p-1] += cards[tmp-1]
        pre[p-1] = tmp
        if cards[tmp-1] == K:
            ans[tmp-1] = turn+1
        if cards[p-1] == K:
            t = p-1
            while t != -1:
                ans[t] = turn+1
                t = pre[t]-1
            s.discard(p)
for a in ans:
    print (a)

9
9
9
15
15
6
-1
-1
6
-1
-1
-1
-1
6
15
